In [1]:
pip install tensorflow keras matplotlib scikit-learn opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 17.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

# Custom function to filter directories
def filter_hidden_dirs(dirnames):
    return [d for d in dirnames if not d.startswith('.')]
    
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Maintain the same validation split
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    classes=filter_hidden_dirs(os.listdir(data_dir))  # Filter hidden directories
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    classes=filter_hidden_dirs(os.listdir(data_dir))  # Filter hidden directories
)

Found 1796 images belonging to 5 classes.
Found 447 images belonging to 5 classes.


In [16]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load the ResNet50 model with ImageNet weights, exclude top layers
base_model = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Create the classification model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(5, activation='softmax')  # 5 output classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


57/57 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.2562 - loss: 15.0783 - val_accuracy: 0.2975 - val_loss: 1.5711
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.3731 - loss: 1.5168 - val_accuracy: 0.3803 - val_loss: 1.6512
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.3144 - loss: 1.7242 - val_accuracy: 0.3445 - val_loss: 1.4842
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 432s 8s/step - accuracy: 0.3584 - loss: 1.6050 - val_accuracy: 0.2729 - val_loss: 1.7731
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.3531 - loss: 1.6174 - val_accuracy: 0.3043 - val_loss: 1.8951
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 356s 6s/step - accuracy: 0.3560 - loss: 1.5281 - val_accuracy: 0.3781 - val_loss: 1.8046
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.3786 - loss: 1.5126 - val_accuracy: 0.3937 - val_loss: 1.5126
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.3887 - loss: 1.5141 - val_accuracy: 0.3557 - val_loss: 1.4555


In [17]:
from tensorflow.keras.layers import BatchNormalization, Dropout

# Enhanced model
enhanced_model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes
])

# Compile the enhanced model
enhanced_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the enhanced model
enhanced_history = enhanced_model.fit(train_data, validation_data=val_data, epochs=20)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.2614 - loss: 2.3280 - val_accuracy: 0.2371 - val_loss: 14.9670
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.3192 - loss: 1.9110 - val_accuracy: 0.1902 - val_loss: 6.3833
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.3303 - loss: 1.7591 - val_accuracy: 0.2729 - val_loss: 4.0389
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.3220 - loss: 1.6865 - val_accuracy: 0.2506 - val_loss: 2.7862
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.3642 - loss: 1.6636 - val_accuracy: 0.2528 - val_loss: 2.0967
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.3780 - loss: 1.5737 - val_accuracy: 0.2998 - val_loss: 1.9356
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.3840 - loss: 1.5136 - val_accuracy: 0.2908 - val_loss: 1.8599
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.4085 - loss: 1.4584 - val_accuracy: 0.2819 - v

In [18]:
from sklearn.metrics import classification_report

# Evaluate on validation data
val_labels = val_data.classes
preds = enhanced_model.predict(val_data)
pred_classes = preds.argmax(axis=-1)

# Generate classification report
print(classification_report(val_labels, pred_classes, target_names=list(train_data.class_indices.keys())))

14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step
              precision    recall  f1-score   support

      RedRot       0.29      0.44      0.35       103
      Mosaic       0.19      0.08      0.11        92
     Healthy       0.27      0.04      0.07       104
        Rust       0.14      0.11      0.12        47
      Yellow       0.25      0.51      0.34       101

    accuracy                           0.25       447
   macro avg       0.23      0.23      0.20       447
weighted avg       0.24      0.25      0.21       447

